## Continue analysis on new reactions' dead-end metabolites
1. add transport for [p] and [e] metabolites
2. break generic reactions into specific reactions
3. subdivide [i] compartment into [p] and [c] compartments

In [1]:
import numpy as np
import ast
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
import networkx as nx
import plotly.express as px

from scipy.special import logsumexp

%matplotlib inline
# sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/dev/vivarium-ecoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
from ecoli.library.schema import numpy_schema, bulk_name_to_idx, listener_schema, counts
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

### Run 20250219_dead_end_metabolite_qual to get the <i> dem_new_clean </i> variable

In [9]:
%store -r dem_new_clean
print(dem_new_clean)
df = pd.DataFrame(index=list(dem_new_clean), columns=['CATEGORY'])
df.to_csv('dem_new.tsv', sep='\t', index=True)

{'CPD-582[c]', 'CcmE-Proteins[c]', 'ADENOSYLCOBALAMIN[c]', 'D-Fructopyranose[p]', 'Carboxylates[c]', 'CPD0-2229[c]', 'CPD-25891[c]', 'CPD0-1118[c]', 'CPD0-2207[c]', 'Alpha-D-aldose-1-phosphates[c]', '2-DEHYDRO-3-DEOXY-D-GLUCONATE[p]', 'Carboxylates[p]', 'ASCORBATE[p]', 'MONOMETHYL-ESTER-OF-TRANS-ACONITATE[c]', 'CPD-18719[p]', 'REDUCED-MENAQUINONE[c]', 'CPD-15978[c]', 'CPD0-2173[p]', 'T2-C4-DECADIENYL-COA[c]', 'CPD-9646[p]', 'CPD-3766[c]', 'ENTB-CPLX[c]', 'ALPHA-RIBAZOLE[c]', 'CPD-9956[c]', 'CPD0-2207[p]', 'CPD-264[c]', 'FERRIC-CITRATE-COMPLEX[e]', 'CPD-20756[p]', 'CPD-3801[p]', '3-KETOBUTYRATE[p]', 'CPD-13853[c]', 'CPD-7836[p]', 'Amino-Acids-20[c]', 'CPD0-2061[c]', 'MALTOSE[e]', 'Ox-Hybrid-Cluster-Proteins[c]', 'CPD-17986[p]', 'CPD0-2114[p]', 'UBIQUINONE-8[c]', 'CPD0-2147[p]', 'CPD-8353[p]', 'CPD-15972[p]', 'CPD0-2241[c]', 'Alcohols[c]', 'D-Ribofuranose[c]', 'Donor-H1[i]', 'CPD0-2485[c]', 'CPD-23445[c]', 'CPD-13852[c]', 'Cytochromes-c[c]', 'CPD0-1185[p]', 'CPD-12587[c]', 'FRU[p]', 'Ccm

In [8]:
list(dem_new_clean)

['CPD-582[c]',
 'CcmE-Proteins[c]',
 'ADENOSYLCOBALAMIN[c]',
 'D-Fructopyranose[p]',
 'Carboxylates[c]',
 'CPD0-2229[c]',
 'CPD-25891[c]',
 'CPD0-1118[c]',
 'CPD0-2207[c]',
 'Alpha-D-aldose-1-phosphates[c]',
 '2-DEHYDRO-3-DEOXY-D-GLUCONATE[p]',
 'Carboxylates[p]',
 'ASCORBATE[p]',
 'MONOMETHYL-ESTER-OF-TRANS-ACONITATE[c]',
 'CPD-18719[p]',
 'REDUCED-MENAQUINONE[c]',
 'CPD-15978[c]',
 'CPD0-2173[p]',
 'T2-C4-DECADIENYL-COA[c]',
 'CPD-9646[p]',
 'CPD-3766[c]',
 'ENTB-CPLX[c]',
 'ALPHA-RIBAZOLE[c]',
 'CPD-9956[c]',
 'CPD0-2207[p]',
 'CPD-264[c]',
 'FERRIC-CITRATE-COMPLEX[e]',
 'CPD-20756[p]',
 'CPD-3801[p]',
 '3-KETOBUTYRATE[p]',
 'CPD-13853[c]',
 'CPD-7836[p]',
 'Amino-Acids-20[c]',
 'CPD0-2061[c]',
 'MALTOSE[e]',
 'Ox-Hybrid-Cluster-Proteins[c]',
 'CPD-17986[p]',
 'CPD0-2114[p]',
 'UBIQUINONE-8[c]',
 'CPD0-2147[p]',
 'CPD-8353[p]',
 'CPD-15972[p]',
 'CPD0-2241[c]',
 'Alcohols[c]',
 'D-Ribofuranose[c]',
 'Donor-H1[i]',
 'CPD0-2485[c]',
 'CPD-23445[c]',
 'CPD-13852[c]',
 'Cytochromes-c[c]